<a href="https://colab.research.google.com/github/sanjay-140114/Sample/blob/master/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# New Section

In [ ]:
!pip install --upgrade openai
from openai import OpenAI
from openai import AsyncOpenAI
import os


In [ ]:
!pip install  python-dotenv
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message.content
    #return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message.content
    #return response.choices[0].message["content"]

In [ ]:
!pip install --upgrade llama-index
!pip install --upgrade pypdf
from llama_index import SimpleDirectoryReader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 1.0 MB/s eta 0:00:00


In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./1. DF Contract_Text_Obfuscated.pdf"]
).load_data()

In [ ]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

16 

<class 'llama_index.schema.Document'>
Doc ID: c202920f-0827-4428-a19c-817b088f0aa4
Text: * The  Companies Wer - Graw Education Contract # 3077 ISBN
AUTHOR AGREEMENT-ROYALTY AGREEMENT, made effective on August 10 2011,
between  School Education Group of The  Companies, Inc.("Publisher"),
a New York corporation, and ("Author"), regarding a commission to be
performed by Author with respect to Publisher'New Core Reading for
National, Fl...


In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What is the royalty text in percentage for the author in United States?"
)
print(str(response))

The royalty text in percentage for the author in the United States is not provided in the given context information.


In [ ]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

Based on Reason and Act principles, calculate Domestic Sales percentage which is Royalty in percentages on Royalty Bearing Components sold in the United States?
What is Foreign percentage which is Royalty in percentages on Royalty Bearing Components sold outside the United States?
What is High Discount 1 which is Discounted Royalty in percentages on Royalty Bearing Components sold outside the United States or inside the United States but not in the school market (i.e., excluding copies sold to schools, school systems, or adult education institutions, but including copies sold to charter schools)?


In [ ]:
# You can try your own question:
new_question = "What is the name of the author?"
eval_questions.append(new_question)

In [ ]:
print(eval_questions)

['Based on Reason and Act principles, calculate Domestic Sales percentage which is Royalty in percentages on Royalty Bearing Components sold in the United States?', 'What is Foreign percentage which is Royalty in percentages on Royalty Bearing Components sold outside the United States?', 'What is High Discount 1 which is Discounted Royalty in percentages on Royalty Bearing Components sold outside the United States or inside the United States but not in the school market (i.e., excluding copies sold to schools, school systems, or adult education institutions, but including copies sold to charter schools)?', 'What is the name of the author?']


In [ ]:
!pip install --upgrade trulens_eval

In [ ]:
!pip install --upgrade utils
import utils

In [ ]:
from utils import build_sentence_window_index
from utils import get_sentence_window_query_engine

ImportError: ignored

In [ ]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

ImportError: ignored

In [ ]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

AttributeError: ignored